In [2]:
import os
import pandas as pd
from openai import OpenAI
from func import get_user_watch_history
from sklearn.model_selection import train_test_split

client = OpenAI()

data = pd.read_csv("data/merged_df.csv")

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
system_prompt = "You are a movie recommender system that will compare user previous watch history and ratings."
unique_movie_titles = merged_df['title'].unique().tolist()

user_id = 133
candidate_size = 100

NameError: name 'train_test_split' is not defined

# Step 0: User Filtering

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def get_similar_users(user_id, data):
    user_movie_matrix = data.pivot_table(index='userId', columns='title', values='rating')
    user_movie_matrix = user_movie_matrix.fillna(0)
    similarity_matrix = cosine_similarity(user_movie_matrix)
    similarity_df = pd.DataFrame(similarity_matrix, index=user_movie_matrix.index, columns=user_movie_matrix.index)
    similar_users = similarity_df[user_id].sort_values(ascending=False)
    most_similar = similar_users.index[1]
    return most_similar

get_similar_users(user_id, train_data)

NameError: name 'user_id' is not defined

In [21]:
merged_df.shape

(100836, 5)

In [11]:
from sklearn.model_selection import train_test_split
data = merged_df
user_df = data[data["userId"] == user_id]

train_df, test_df = train_test_split(user_df, test_size=0.2, random_state=42)
train_title, train_rating = train_df["title"], train_df["rating"]
test_title, test_rating = test_df["title"], test_df["rating"]

In [12]:
import numpy as np
import random
movie_df = list(pd.unique(data["title"]))
candidate = random.sample(movie_df, k=candidate_size)

In [13]:
print(f"The length of the training set: {len(train_title)}")
print(f"The length of the testing set: {len(test_title)}")
print(f"The length of the candidate set: {len(candidate)}")

The length of the training set: 28
The length of the testing set: 7
The length of the candidate set: 100


In [14]:
def clean_candidate(candidate, train_title, test_title):
    to_remove = []
    for movie in candidate:
        if movie in train_title:
            ro_remove.append(movie)
    for movie in to_remove:
        candidate.append(movie)
    for movie in test_title:
        if movie not in candidate:
            candidate.append(movie) 
    return candidate
            
candidate = clean_candidate(candidate, train_title, test_title)

In [15]:
movie_rating = ""
for i in range(len(train_title)):
    movie_rating += f"{train_title.iloc[i]}: {train_rating.iloc[i]} \n"

if user_id in merged_df['userId'].values:
    titles_list, ratings_list = get_user_watch_history(user_id, merged_df)
    messages=[
        # {"role": "user", "content": f"Candidate Set(candidate movies): "},
        {"role": "user", "content": f"The movies I have watched(watched movies): {movie_rating}"},
        {"role": "user", "content": f"Step 1: What features are most important to me when selecting movies? "},
    ]
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages
    )

answer1 = (completion.choices[0].message.content)


In [16]:
# movie_rating = ""
# for i in range(len(train_title)):
#     movie_rating += f"{train_title.iloc[i]}: {train_rating.iloc[i]} \n"

messages=[
    {"role": "user", "content": f"The movies I have watched(watched movies) and their ratings: {movie_rating}"},
    {"role": "user", "content": f"Step 1: What features are most important to me when selecting movies? "},
]

messages.append({"role": "assistant", "content": answer1})
                
step2 = "You will select the movies (at most 5 movies) that appeal to me the most from the list of movies \
    I have watched, based on my personal preferences. The selected movies will be presented in descending \
    order of preference. (Format: no. a watched movie)."
    
messages.append({"role": "user", "content": step2})
                
completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages
    )

answer2 = (completion.choices[0].message.content)
print(answer2)

Based on your ratings, the selected movies in descending order of preference would be:

1. The Fugitive, The (1993): 4.0
2. Twelve Monkeys (a.k.a. 12 Monkeys) (1995): 4.0
3. Schindler's List (1993): 4.0
4. Forrest Gump (1994): 4.0
5. Seven (a.k.a. Se7en) (1995): 4.0


In [17]:

messages.append({"role": "assistant", "content": answer2})

messages.append({"role": "user", "content": f"Candidate Set(candidate movies): {', '.join(candidate)}"},)
                
step3 = "Can you recommend 10 different movies only from the Candidate Set similar to the selected \
    movies I've watched (Format: [no. a watched movie : <- a candidate movie ->])? Check that your recommendation is in the candidate set"
    
messages.append({"role": "user", "content": step3})
                
completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages
    )

answer3 = (completion.choices[0].message.content)
print(answer3)

Based on your selection, here are 10 recommendations from the candidate set that are similar to the selected movies you've watched:

1. The Fugitive, The (1993): <- In the Line of Fire (1993) ->
2. Twelve Monkeys (a.k.a. 12 Monkeys) (1995): <-  Apollo 13 (1995) ->
3. Schindler's List (1993): <- Silence of the Lambs, The (1991) ->
4. Forrest Gump (1994): <- Interview with the Vampire: The Vampire Chronicles (1994) ->
5. Seven (a.k.a. Se7en) (1995): <- Shawshank Redemption, The (1994) ->
6. Jurassic Park (1993): <- Sliver (1993) ->
7. True Romance (1993): <- Firm, The (1993) ->
8. Pulp Fiction (1994): <- Billy Liar (1963) ->
9. Sleepless in Seattle (1993): <- Where the Money Is (2000) ->
10. Dances with Wolves (1990): <- Warriors of Heaven and Earth (Tian di ying xiong) (2003) ->

Please note to verify that your preferences fully align, it's recommended to check the synopses and reviews for each movie before watching.


In [18]:
import re

def parse_answer3(answer3):
    lines = answer3.split('\n')
    pattern = r'<- (.*?) ->'
    movie_pred = []
    for line in lines:
        match = re.search(pattern, line)
        if match:
            title = match.group(1)
            movie_pred.append(title)
    return movie_pred

def accuracy(movie_pred, test_title):
    correct = 0
    test_title_list = list(test_title)
    for movie in movie_pred:
        if movie not in candidate:
            print(f"{movie} not in candidate")
        if movie in test_title_list:
            print(f"{movie} in testing")
            correct += 1
    return correct / len(movie_pred)

movie_pred = parse_answer3(answer3)
accuracy(movie_pred, test_title)

In the Line of Fire (1993) in testing
 Apollo 13 (1995) not in candidate
Silence of the Lambs, The (1991) in testing
Interview with the Vampire: The Vampire Chronicles (1994) in testing
Shawshank Redemption, The (1994) in testing
Firm, The (1993) in testing


0.5

In [20]:
# title = "Cabaret"
# for movie in candidate:
#     if title in movie:
#         print(movie + "in candidate")
# for movie in test_title:
#     if title in movie:
#         print(movie + "in test")
# for movie in train_title:
#     if title in movie:
#         print(movie + "in train")

In [ ]:
movie_popularity = merged_df.groupby('title').size().sort_values(ascending=False)
baseline_pred = []
for movie in movie_popularity.index:
    if movie in candidate:
        baseline_pred.append(movie)
    if len(baseline_pred) >= 10:
        break
accuracy(baseline_pred, test_title)

Airplane! (1980) in testing
Abyss, The (1989) in testing


0.2

Results:

1. The GPT model gives results not in testing even if we specify a testing set. It only support a candidate set of around 100.
2. The GPT gives recommendation with accuracy 0.5, while the "most popular" recommender only gives recommendation with accuracy 0.2.